##### To Do - Updated on 21st July 10:32AM
##### Experiment with shorter windows

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import pi #Used for some experimentation purposes
from scipy.fftpack import fft
import seaborn as sns
from tslearn.metrics import dtw, dtw_path
import scipy.signal
from cmath import rect,pi

In [4]:
data=pd.read_csv('./BrahmsConcertPart1/Keypoint Data/BrahmsConcertPart1_File5.csv',index_col=0)

## Create duplicate of data

In [5]:
df=data.copy()

## To be used when computing for the three parts

In [106]:
first, second, third = np.split(df, [int((1/3)*len(df)), int((2/3)*len(df))])
first.reset_index(drop=True,inplace=True)
second.reset_index(drop=True,inplace=True)
third.reset_index(drop=True,inplace=True)

##### To be chosen based on experiment. If experimenting for the entire phrase, skip.

In [107]:
df=first

In [116]:
df=second

In [125]:
df=third

##### Time under observation

In [6]:
df['t'].max()

19.288135593220503

#### To be edited here before experiments: Window size (Difference between a & b) and Shift

In [7]:
d={}
a=0
b=149
shift=15
count=1
count_start=a+b+1
count_for_window=count_start
while(count_start!=len(df)):

    count+=1
    count_start+=shift

for i in range(count):
    df1=df.loc[a:b]
    d["group"+str(i)]=df1
    a+=shift
    b+=shift

#### Window Size

In [8]:
count

29

#### Functions for processing data

In [9]:
def getWindowData(a):
    pct={}
    for i in range(count):
        pct['window'+str(i)]=d['group'+str(i)][[a,'t']]
    return pct

In [10]:
def getFFT(a):
    N=len(a)
    sample_rate=30
    waveform=np.array(a)
    freq_data = fft(waveform)
    y = (2/N) * np.abs (freq_data [0:np.int (N/2)])
    y_angle= np.angle(freq_data,deg=False)
    y_ampl= np.abs(freq_data)
    df_pct_ang_amp=pd.DataFrame({'Angle':y_angle,'Amplitude':y_ampl})
    return df_pct_ang_amp

In [11]:
def getFFTDict(pct,pct_data):
    #person1/person2 are participants
    #person1_aa is participant angle and amplitude data of person1 and likewise for person2_aa
    dict_paa={}
    for i in range(count):
        df=getFFT(pct_data['window'+str(i)][pct+'_d'])
        dict_paa['window'+str(i)]=df
    return dict_paa

In [12]:
def rel_angle(l1,l2):
    rel_angledict={}
    for i in range(count):
        rel_anglex=l1['window'+str(i)]['Angle']-l2['window'+str(i)]['Angle']
        rel_angledict['window'+str(i)]=rel_anglex
    return rel_angledict

##### Automated - Set limits to 1:51 for a window size of 150 and 1:11 for a window size of 30

In [15]:
for i in range(5):
    for j in range(5):
        participant1='c'+str(i+1)
        participant2='c'+str(j+1)
        if ((participant1!=participant2)&(j>i)):
            person1=getWindowData(participant1+'_d')
            person2=getWindowData(participant2+'_d')
            person1_aa=getFFTDict(participant1,person1)
            person2_aa=getFFTDict(participant2,person2)

            x=rel_angle(person1_aa,person2_aa)

            plv_results={}

            for k in range(count_for_window):
                my_list = [elem[k] for elem in x.values()]
                rel_amp=[1]*len(my_list)
                rel_sum = [x for x in map(rect,rel_amp,my_list)]
                plv_results[str(k)]=abs(sum(rel_sum))/(len(rel_sum))

            plv_results1=list(plv_results.values())
#             print((sum(plv_results1[1:11])/len(plv_results1[1:11])))
            print("Dyadic Synchronization between ",participant1," and ",participant2,":",(sum(plv_results1[1:11])/len(plv_results1[1:11])))

Dyadic Synchronization between  c1  and  c2 : 0.44341003967654846
Dyadic Synchronization between  c1  and  c3 : 0.41628189678901883
Dyadic Synchronization between  c1  and  c4 : 0.43298126623339944
Dyadic Synchronization between  c1  and  c5 : 0.42757835442712
Dyadic Synchronization between  c2  and  c3 : 0.41511063040380936
Dyadic Synchronization between  c2  and  c4 : 0.34887861976951406
Dyadic Synchronization between  c2  and  c5 : 0.41859387292646766
Dyadic Synchronization between  c3  and  c4 : 0.272738366600198
Dyadic Synchronization between  c3  and  c5 : 0.2958538461901187
Dyadic Synchronization between  c4  and  c5 : 0.3393011973081942


##### For Manual execution. Can Skip

In [34]:
participant1='c1'
participant2='c2'

person1=getWindowData(participant1+'_d')
person2=getWindowData(participant2+'_d')

person1_aa=getFFTDict(participant1,person1)
person2_aa=getFFTDict(participant2,person2)

x=rel_angle(person1_aa,person2_aa)

plv_results={}

for i in range(150):
    my_list = [elem[i] for elem in x.values()]
    rel_amp=[1]*len(my_list)
    rel_sum = [x for x in map(rect,rel_amp,my_list)]
    plv_results[str(i)]=abs(sum(rel_sum))/(len(rel_sum))

plv_results1=list(plv_results.values())

In [35]:
len(my_list)

29

In [36]:
x

{'window0': 0      0.000000
 1     -4.374689
 2     -0.692991
 3      4.147685
 4     -1.321356
          ...   
 145    1.381419
 146    1.321356
 147   -4.147685
 148    0.692991
 149    4.374689
 Name: Angle, Length: 150, dtype: float64,
 'window1': 0      0.000000
 1     -4.655705
 2     -2.062443
 3     -3.103837
 4     -0.534293
          ...   
 145    1.858598
 146    0.534293
 147    3.103837
 148    2.062443
 149    4.655705
 Name: Angle, Length: 150, dtype: float64,
 'window2': 0      0.000000
 1      1.338976
 2     -2.722491
 3     -2.424151
 4     -1.168734
          ...   
 145    1.412041
 146    1.168734
 147    2.424151
 148    2.722491
 149   -1.338976
 Name: Angle, Length: 150, dtype: float64,
 'window3': 0      0.000000
 1      1.007631
 2     -4.143112
 3      4.190837
 4     -1.191984
          ...   
 145    1.393644
 146    1.191984
 147   -4.190837
 148    4.143112
 149   -1.007631
 Name: Angle, Length: 150, dtype: float64,
 'window4': 0      0.000000
 1      

In [37]:
my_list

[4.374689011794249,
 4.655705289333376,
 -1.338975957381278,
 -1.0076310192506717,
 -0.7515639677062651,
 -0.7485155781080471,
 -0.6478387107108058,
 -1.0483593404345177,
 -2.738323662441876,
 -3.0839193983629403,
 -1.246179597316803,
 -1.571425509269092,
 -1.5752353229766982,
 -2.004655906952301,
 3.9740280811744637,
 3.4309440445319286,
 3.476731914453704,
 4.018712092206263,
 -2.32300244617337,
 -2.404525700721151,
 -2.414672844995825,
 -3.008512422850212,
 -2.535522249392646,
 -2.0374098463602013,
 -1.3791594525993895,
 -0.6120203690940884,
 0.14545659987419057,
 0.14914017725615025,
 0.49206401914212483]

In [25]:
rel_amp

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [28]:
len(my_list)

29

In [24]:
rel_sum

[(-0.6156622437597655-0.7880101532395957j),
 (-0.7882211230816083-0.6153921198130246j),
 (-0.9572292665641784+0.28933048790786836j),
 (0.9825099789486633+0.1862099386882912j),
 (0.3109324321533506+0.9504320189446492j),
 (0.9994252452609682+0.033899544761744595j),
 (0.8386069103265006+0.5447370466129878j),
 (-0.032044291742317745-0.9994864498164712j),
 (0.5698630910718069-0.8217396530739438j),
 (-0.22466714621705172-0.9744355665772293j),
 (0.6197938697763072-0.7847646519738959j),
 (0.8711535122365373+0.49101075152988793j),
 (0.6750803449195651+0.7377442157708055j),
 (0.855605759678558+0.5176280363396843j),
 (0.9614938679179552-0.2748263851164398j),
 (0.6246990888752946-0.7808655763691833j),
 (0.5691204311136338-0.8222541790036897j),
 (0.6985189115672094-0.7155915945446545j),
 (0.9662753052629928-0.2575112316752613j),
 (0.8912942351570123-0.4534253922927964j),
 (0.7573396297521954-0.6530211981297449j),
 (0.4377210533661603-0.8991108271175579j),
 (-0.3054020721963881-0.9522234896798925j),

In [23]:
plv_results1

[1.0,
 0.6095626397864132,
 0.2303861988979932,
 0.5477782308910112,
 0.4068922536760999,
 0.506852601228461,
 0.24272206616416486,
 0.6051370268667063,
 0.573762495268401,
 0.3473698083427609,
 0.36363707564347264,
 0.34584523377678333,
 0.1708140649542447,
 0.22552326548930088,
 0.2862178925030214,
 0.6410540074601627,
 0.36335072875202246,
 0.3966272280498231,
 0.2525718876668975,
 0.3481247141052962,
 0.3299137211254991,
 0.27022426671255456,
 0.4011178951422974,
 0.08681351000498078,
 0.25801239393548037,
 0.22961362946644207,
 0.5285676288207943,
 0.4293225773197312,
 0.5857503817340823]

In [12]:
print("Synchronisation Index:",sum(plv_results1[1:11])/len(plv_results1[1:11]))

Synchronisation Index: 0.16317359621908215


In [13]:
sum(rel_sum)

(2.3530444171692713+3.9600031438897156j)

In [38]:
x

{'window0': 0      0.000000
 1     -4.374689
 2     -0.692991
 3      4.147685
 4     -1.321356
          ...   
 145    1.381419
 146    1.321356
 147   -4.147685
 148    0.692991
 149    4.374689
 Name: Angle, Length: 150, dtype: float64,
 'window1': 0      0.000000
 1     -4.655705
 2     -2.062443
 3     -3.103837
 4     -0.534293
          ...   
 145    1.858598
 146    0.534293
 147    3.103837
 148    2.062443
 149    4.655705
 Name: Angle, Length: 150, dtype: float64,
 'window2': 0      0.000000
 1      1.338976
 2     -2.722491
 3     -2.424151
 4     -1.168734
          ...   
 145    1.412041
 146    1.168734
 147    2.424151
 148    2.722491
 149   -1.338976
 Name: Angle, Length: 150, dtype: float64,
 'window3': 0      0.000000
 1      1.007631
 2     -4.143112
 3      4.190837
 4     -1.191984
          ...   
 145    1.393644
 146    1.191984
 147   -4.190837
 148    4.143112
 149   -1.007631
 Name: Angle, Length: 150, dtype: float64,
 'window4': 0      0.000000
 1      

In [63]:
len(plv_results1)

30